# Environmental Impact Evaluation of Concrete Frame Bridges

This notebook takes the parameter vectors x as input and calculates first the volumes and the environmental impacts of all the structures.

In [1]:
import pickle
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import math as m

C:\Users\kuhnso\AppData\Local\Temp\ipykernel_3564\1637765061.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
idx_s=800


# Import CFM_xy data
xy_file_name='CFBData\\{}_Batch\\{}_CFB_xy.csv'.format(idx_s,idx_s)

df_xy=pd.read_csv(xy_file_name)
df_xy

,Unnamed: 0,ID,L,b1,t_p,t_w,h_w,d1_plate,d4_plate,d2_plate,...,y_s,z_s,Location_s,GP_count_s,eta_min_c,x_c,y_c,z_c,Location_c,GP_count_c
0,0,0,5896.042103,10183.791863,1189.025419,587.175975,3951.288982,16.589760,24.595059,15.124578,...,2463.710938,-1140.493774,eta_1_x,0.0,41.178042,9817.936523,245.655258,-3552.487061,eta_3_top,0.0
1,1,1,9783.893188,15582.253981,434.089655,668.134731,4157.543342,10.005126,15.433241,11.179018,...,4856.115234,-394.626953,eta_1_x,52.0,0.007260,14298.742188,4259.643555,-394.626953,eta_3_top,433.0
2,2,2,3518.491402,15074.031207,509.342737,453.005723,4833.272653,19.790377,21.281638,25.048946,...,181.202286,-2277.959229,eta_1_x,0.0,7.161654,12240.676758,181.202286,-4713.654297,eta_3_bot,0.0
3,3,3,8975.183339,9902.815547,562.704986,824.904330,2401.438242,29.192119,13.684717,22.211689,...,7866.090332,-514.815186,eta_4_x,0.0,5.194534,345.297424,5167.886230,-514.815186,eta_3_top,0.0
4,4,3,8975.183339,9902.815547,562.704986,824.904330,2401.438242,29.192119,13.684717,22.211689,...,7866.090332,-514.815186,eta_4_x,0.0,5.194534,345.297424,5167.886230,-514.815186,eta_3_top,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758,2758,2136,14265.476237,19725.889956,1009.441111,770.200223,3165.283883,14.807158,24.389714,25.670920,...,8557.013672,-962.851501,eta_1_x,0.0,3.425795,19253.451172,13157.573242,-1833.246826,eta_3_bot,0.0
2759,2759,2137,3065.223231,13240.759972,943.505108,393.137989,4831.098192,11.227795,14.475960,22.927705,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2760,2760,2138,14108.767796,15006.354086,826.971868,1163.631750,3117.787627,17.494606,12.873865,18.204315,...,11260.567383,-779.031494,eta_4_x,0.0,2.596107,14646.949219,4736.952148,-779.031494,eta_3_top,0.0
2761,2761,2139,4165.437981,14000.795500,224.499114,661.635841,3166.214513,20.458322,26.246573,20.339736,...,1282.129761,-177.236145,eta_1_x,0.0,6.182369,9366.270508,1626.026245,-177.236145,eta_3_top,0.0


In [3]:
df_xy['eta_min_s']

0       9.049629
1       0.866557
2       1.610580
3       2.073693
4       2.073693
          ...   
2758    2.107467
2759    0.000000
2760    2.167530
2761    3.706076
2762    1.576030
Name: eta_min_s, Length: 2763, dtype: float64

In [4]:
df_xy['eta_min_tot']=df_xy[['eta_min_c','eta_min_s']].min(axis=1)
df_xy['eta_min_tot']

0       9.049629
1       0.007260
2       1.610580
3       2.073693
4       2.073693
          ...   
2758    2.107467
2759    0.000000
2760    2.167530
2761    3.706076
2762    0.018379
Name: eta_min_tot, Length: 2763, dtype: float64

In [5]:
df_xy['d4_walls']

0       14.595347
1       19.672999
2       29.445393
3       19.976230
4       19.976230
          ...    
2758    13.731583
2759    25.869709
2760    28.472067
2761    25.553295
2762    27.334584
Name: d4_walls, Length: 2763, dtype: float64

In [6]:
# start_ID=0
# end_ID=19
# ID=1

##### Calculate Volumes of Deck Slab of concrete and steel
#volume of Concrete
vol_plate_conc=df_xy['t_p']/1000 * df_xy['L']/1000 * (df_xy['b1'] + df_xy['b2'])/2/1000 #[m3]

#volume of Reinforcement
As_1=((df_xy['d1_plate']/1000)**2)*m.pi/4
n_1=((df_xy['b1']+df_xy['b2'])/2)/df_xy['s_plate']

As_4=((df_xy['d4_plate']/1000)**2)*m.pi/4
n_4=((df_xy['b1']+df_xy['b2'])/2)/df_xy['s_plate']

As_2=((df_xy['d2_plate']/1000)**2)*m.pi/4
n_2=df_xy['L']/df_xy['s_plate']

As_3=((df_xy['d3_plate']/1000)**2)*m.pi/4
n_3=df_xy['L']/df_xy['s_plate']

vol_plate_steel = (As_1*n_1 + As_4*n_4)*df_xy['L']/1000  +  (As_2*n_2+As_3*n_3) * ((df_xy['b1']+df_xy['b2'])/2)/1000 #[m3]
w_plate_steel = vol_plate_steel* 7850   #[kg]      #Unit weight of steel is taken as 7850 kg/m3



##### Calculate Volumes of both walls of concrete and steel
#volume of Concrete
vol_walls_conc=df_xy['t_w']/1000 * df_xy['h_w']/1000 *  (df_xy['b1']+df_xy['b2'])/1000     #[m3]



#volume of Reinforcement
As_1=((df_xy['d1_walls']/1000)**2)*m.pi/4
n_1=((df_xy['b1']+df_xy['b2'])/2)/df_xy['s_walls']

As_4=((df_xy['d4_walls']/1000)**2)*m.pi/4
n_4=((df_xy['b1']+df_xy['b2'])/2)/df_xy['s_walls']

As_2=((df_xy['d2_walls']/1000)**2)*m.pi/4
n_2=df_xy['L']/df_xy['s_walls']

As_3=((df_xy['d3_walls']/1000)**2)*m.pi/4
n_3=df_xy['L']/df_xy['s_walls']

vol_walls_steel = ((As_1*n_1 + As_4*n_4)*df_xy['h_w']/1000)*2  +  (As_2*n_2 + As_3*n_3) * (df_xy['b1']+df_xy['b2'])/1000 #[m3]
w_walls_steel = vol_walls_steel* 7850   #[kg]      #Unit weight of steel is taken as 7850 kg/m3


matQuantity_dict={'vol_plate_conc':vol_plate_conc, 'vol_plate_steel':vol_plate_steel,'w_plate_steel':w_plate_steel,
                    'vol_walls_conc':vol_walls_conc, 'vol_walls_steel':vol_walls_steel, 'w_walls_steel':w_walls_steel }

df_matQ=pd.DataFrame(matQuantity_dict)

df_xyz = pd.concat([df_xy, df_matQ], axis=1)
df_xyz


,Unnamed: 0,ID,L,b1,t_p,t_w,h_w,d1_plate,d4_plate,d2_plate,...,z_c,Location_c,GP_count_c,eta_min_tot,vol_plate_conc,vol_plate_steel,w_plate_steel,vol_walls_conc,vol_walls_steel,w_walls_steel
0,0,0,5896.042103,10183.791863,1189.025419,587.175975,3951.288982,16.589760,24.595059,15.124578,...,-3552.487061,eta_3_top,0.0,9.049629,71.393920,0.135610,1064.539985,47.254871,0.220156,1728.223765
1,1,1,9783.893188,15582.253981,434.089655,668.134731,4157.543342,10.005126,15.433241,11.179018,...,-394.626953,eta_3_top,433.0,0.007260,66.179186,0.183277,1438.721753,86.568742,0.451796,3546.596502
2,2,2,3518.491402,15074.031207,509.342737,453.005723,4833.272653,19.790377,21.281638,25.048946,...,-4713.654297,eta_3_bot,0.0,1.610580,27.014443,0.521039,4090.154739,66.009188,0.286251,2247.068987
3,3,3,8975.183339,9902.815547,562.704986,824.904330,2401.438242,29.192119,13.684717,22.211689,...,-514.815186,eta_3_top,0.0,2.073693,50.012986,0.297289,2333.717250,39.234100,0.364146,2858.544124
4,4,3,8975.183339,9902.815547,562.704986,824.904330,2401.438242,29.192119,13.684717,22.211689,...,-514.815186,eta_3_top,0.0,2.073693,50.012986,0.297289,2333.717250,39.234100,0.364146,2858.544124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758,2758,2136,14265.476237,19725.889956,1009.441111,770.200223,3165.283883,14.807158,24.389714,25.670920,...,-1833.246826,eta_3_bot,0.0,2.107467,284.055936,1.370487,10758.322714,96.179587,1.360062,10676.488963
2759,2759,2137,3065.223231,13240.759972,943.505108,393.137989,4831.098192,11.227795,14.475960,22.927705,...,NaN,NaN,NaN,0.000000,38.292990,0.108498,851.711304,50.296039,0.262115,2057.601601
2760,2760,2138,14108.767796,15006.354086,826.971868,1163.631750,3117.787627,17.494606,12.873865,18.204315,...,-779.031494,eta_3_top,0.0,2.167530,175.087448,0.480137,3769.077565,108.884805,0.408871,3209.634196
2761,2761,2139,4165.437981,14000.795500,224.499114,661.635841,3166.214513,20.458322,26.246573,20.339736,...,-177.236145,eta_3_top,0.0,3.706076,13.092664,0.225797,1772.504185,58.660001,0.339092,2661.873675


In [7]:
### define environmental impact factors of my material

materials_concrete = {  'C30/37_average': {'Description': 'Durchschnittliche Zusammensetzungen für Beton in Deutschland (compression strength of 30 MPa) vom InformationsZentrum Beton GmbH',
                                         'EpdLink': 'https://www.oekobaudat.de/OEKOBAU.DAT/resource/sources/dc600cd9-9a55-414e-baf5-f36c54269803/Beton_der_Druckfestigkeitsklasse_C_3037_10616.pdf?version=00.02.000',
                                          'AP': 0.3431,
                                          'GWP': 218.71,
                                          'ADP-fossil': 1043.1 ,
                                          'ADP-minerals-metals': 0.000705978,
                                          'ODP': 0.000000059584079,
                                          'POCP': 0.016064, 
                                         },
                 
                        'C45/55_average': {'Description': 'durchschnittliche Zusammensetzungen für Beton in Deutschland (compression strength of 30 MPa) vom InformationsZentrum Beton GmbH',
                                         'EpdLink': 'https://www.oekobaudat.de/OEKOBAU.DAT/resource/sources/ef01ced5-dd04-43b8-80ce-5a5a7c421657/Beton_der_Druckfestigkeitsklasse_C_4555.pdf?version=00.01.000',
                                          'AP': 0.4321,
                                          'GWP': 285.71 ,
                                          'ADP-fossil': 1406.1 ,
                                          'ADP-minerals-metals': 0.001014978,
                                          'ODP': 0.000000077084079,
                                          'POCP': 0.025364, 
                                         },

                        'C50/60_average': {'Description': 'durchschnittliche Zusammensetzungen für Beton in Deutschland (compression strength of 30 MPa) vom InformationsZentrum Beton GmbH',
                                         'EpdLink': 'https://www.oekobaudat.de/OEKOBAU.DAT/resource/sources/892ca341-33e1-4136-81b9-ccd8418e9d73/Beton_der_Druckfestigkeitsklasse_C_5060_10526.pdf?version=00.03.000',
                                          'AP': 0.4481,
                                          'GWP': 299.71,
                                          'ADP-fossil': 1406.1 ,
                                          'ADP-minerals-metals': 0.001064978,
                                          'ODP': 0.000000083884079,
                                          'POCP': 0.028164, 
                                         }
                        }


materials_reinforcement= {  'Rebar Stahl Gerlafingen': {'Description': 'Rebar produced at Gerlafingen (Switzerland) B500B',
                                         'EpdLink': 'https://gruppobeltrame.com/wp-content/uploads/2021/02/EPD_Rebar-TOPAR-RC-rev1.pdf',
                                          'AP': 0.0009637,
                                          'GWP': 0.25054,
                                          'ADP-fossil': 9.8661,
                                          'ADP-minerals-metals': -0.0000015945439,
                                          'ODP': 0.000000037507,
                                          'POCP': 0.00078423, 
                                         }
                        }

In [8]:
# calc gwp
def calc_gwp(row):
    fcc=row['fcc']

    if fcc < 37.5:
        gwp_fac_conc = materials_concrete['C30/37_average']['GWP']
    elif 37.5 <= fcc < 47.5:
        gwp_fac_conc = materials_concrete['C45/55_average']['GWP']
    elif fcc >= 47.5:
        gwp_fac_conc = materials_concrete['C50/60_average']['GWP']
    else:
        raise ValueError(f"fcc value '{fcc}' does not fit the defined ranges.")
    
    gwp_fac_steel=materials_reinforcement['Rebar Stahl Gerlafingen']['GWP']

    gwp=gwp_fac_conc*(row['vol_plate_conc']+row['vol_walls_conc']) + gwp_fac_steel*(row['w_plate_steel']+row['w_walls_steel'])

    return gwp

In [9]:
# Calculate environemntal impact
def calc_envImpact(row, envImpact):
    fcc=row['fcc']

    if fcc < 37.5:
        eI_fac_conc = materials_concrete['C30/37_average'][envImpact]
    elif 37.5 <= fcc < 47.5:
        eI_fac_conc = materials_concrete['C45/55_average'][envImpact]
    elif fcc >= 47.5:
        eI_fac_conc = materials_concrete['C50/60_average'][envImpact]
    else:
        raise ValueError(f"fcc value '{fcc}' does not fit the defined ranges.")
    
    eI_fac_steel=materials_reinforcement['Rebar Stahl Gerlafingen'][envImpact]

    eI=eI_fac_conc*(row['vol_plate_conc']+row['vol_walls_conc']) + eI_fac_steel*(row['w_plate_steel']+row['w_walls_steel'])

    return eI

In [10]:
df_xyz['GWP']=df_xyz.apply(lambda row: calc_envImpact(row, envImpact='GWP'), axis=1)
df_xyz['GWP']

0       34598.845160
1       44890.632059
2       21932.926394
3       28049.113122
4       28049.113122
            ...     
2758    88531.588877
2759    26039.670694
2760    82882.158676
2761    21611.442890
2762    48683.968164
Name: GWP, Length: 2763, dtype: float64

In [11]:
df_xyz['AP']=df_xyz.apply(lambda row: calc_envImpact(row, 'AP'), axis=1)
df_xyz['AP']

0        53.959529
1        70.806731
2        38.023590
3        44.995401
4        44.995401
           ...    
2758    151.115536
2759     41.083024
2760    129.429795
2761     35.277736
2762     78.484664
Name: AP, Length: 2763, dtype: float64

In [12]:
df_xyz['ADP-minerals-metals']=df_xyz.apply(lambda row: calc_envImpact(row, 'ADP-minerals-metals'), axis=1)
df_xyz['ADP-minerals-metals']

0       0.115973
1       0.147086
2       0.055568
3       0.086767
4       0.086767
          ...   
2758    0.234259
2759    0.085277
2760    0.277098
2761    0.065757
2762    0.152316
Name: ADP-minerals-metals, Length: 2763, dtype: float64

In [13]:
df_xyz['ADP-fossil']=df_xyz.apply(lambda row: calc_envImpact(row, 'ADP-fossil'), axis=1)
df_xyz['ADP-fossil']

0       194385.751723
1       263964.509574
2       159556.632616
3       176717.696686
4       176717.696686
            ...      
2758    608101.669168
2759    153268.605543
2760    468146.052197
2761    144641.437381
2762    311287.474989
Name: ADP-fossil, Length: 2763, dtype: float64

In [14]:
df_xyz['ODP']=df_xyz.apply(lambda row: calc_envImpact(row, 'ODP'), axis=1)
df_xyz['ODP']

0       0.000114
1       0.000199
2       0.000243
3       0.000202
4       0.000202
          ...   
2758    0.000827
2759    0.000116
2760    0.000284
2761    0.000172
2762    0.000324
Name: ODP, Length: 2763, dtype: float64

In [15]:
df_xyz['POCP']=df_xyz.apply(lambda row: calc_envImpact(row, 'POCP'), axis=1)
df_xyz['POCP']

0        5.199577
1        7.783935
2        6.464173
3        6.585482
4        6.585482
          ...    
2758    22.917926
2759     4.528543
2760    12.675587
2761     5.297507
2762    10.649829
Name: POCP, Length: 2763, dtype: float64

In [16]:
df_xyz

,Unnamed: 0,ID,L,b1,t_p,t_w,h_w,d1_plate,d4_plate,d2_plate,...,w_plate_steel,vol_walls_conc,vol_walls_steel,w_walls_steel,GWP,AP,ADP-minerals-metals,ADP-fossil,ODP,POCP
0,0,0,5896.042103,10183.791863,1189.025419,587.175975,3951.288982,16.589760,24.595059,15.124578,...,1064.539985,47.254871,0.220156,1728.223765,34598.845160,53.959529,0.115973,194385.751723,0.000114,5.199577
1,1,1,9783.893188,15582.253981,434.089655,668.134731,4157.543342,10.005126,15.433241,11.179018,...,1438.721753,86.568742,0.451796,3546.596502,44890.632059,70.806731,0.147086,263964.509574,0.000199,7.783935
2,2,2,3518.491402,15074.031207,509.342737,453.005723,4833.272653,19.790377,21.281638,25.048946,...,4090.154739,66.009188,0.286251,2247.068987,21932.926394,38.023590,0.055568,159556.632616,0.000243,6.464173
3,3,3,8975.183339,9902.815547,562.704986,824.904330,2401.438242,29.192119,13.684717,22.211689,...,2333.717250,39.234100,0.364146,2858.544124,28049.113122,44.995401,0.086767,176717.696686,0.000202,6.585482
4,4,3,8975.183339,9902.815547,562.704986,824.904330,2401.438242,29.192119,13.684717,22.211689,...,2333.717250,39.234100,0.364146,2858.544124,28049.113122,44.995401,0.086767,176717.696686,0.000202,6.585482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758,2758,2136,14265.476237,19725.889956,1009.441111,770.200223,3165.283883,14.807158,24.389714,25.670920,...,10758.322714,96.179587,1.360062,10676.488963,88531.588877,151.115536,0.234259,608101.669168,0.000827,22.917926
2759,2759,2137,3065.223231,13240.759972,943.505108,393.137989,4831.098192,11.227795,14.475960,22.927705,...,851.711304,50.296039,0.262115,2057.601601,26039.670694,41.083024,0.085277,153268.605543,0.000116,4.528543
2760,2760,2138,14108.767796,15006.354086,826.971868,1163.631750,3117.787627,17.494606,12.873865,18.204315,...,3769.077565,108.884805,0.408871,3209.634196,82882.158676,129.429795,0.277098,468146.052197,0.000284,12.675587
2761,2761,2139,4165.437981,14000.795500,224.499114,661.635841,3166.214513,20.458322,26.246573,20.339736,...,1772.504185,58.660001,0.339092,2661.873675,21611.442890,35.277736,0.065757,144641.437381,0.000172,5.297507


In [17]:
xyz_file_name='CFBData\\{}_Batch\\{}_CFB_xyz.csv'.format(idx_s,idx_s)
df_xyz.to_csv(xyz_file_name)